In [23]:
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm

import warnings
warnings.filterwarnings("ignore")

In [24]:
data_real = pd.read_csv('processed_data/ProcessedTrainReal//ProcessedTrain.csv.gz')
data_real = data_real[data_real['edepR'] < 17.2]

In [25]:
N = int(5e4)

X = data_real.iloc[:, :-5]
y = data_real.iloc[:, -5]

X_train = X[:N]
y_train = y[:N]

X_val = X[N:]
y_val = y[N:]

In [26]:
# import pickle
#
# xgbreg = XGBRegressor(
#         max_depth=9,
#         learning_rate=0.08,
#         n_estimators=3000,
#         random_state=22,
# )

# xgbreg.fit(X_train, y_train, verbose=False,
#            eval_set=[(X_val, y_val)],
#            early_stopping_rounds=7)

# pickle.dump(xgbreg, open("models/xgb_energy_real_fs.dat", "wb"))

In [27]:
import pickle
xgbreg = pickle.load(open("models/xgb_energy_real.dat", "rb"))
y_predict = xgbreg.predict(X_val)

In [28]:
from sklearn.metrics import mean_squared_error
all_features_rmse = mean_squared_error(y_val, y_predict)**0.5
all_features_rmse

0.0723105782057552

In [29]:
opt_features = ['AccumCharge']
current_rmses = []
current_rmse = 100

features = X_train.columns
features.drop('AccumCharge')
while abs(all_features_rmse - current_rmse) > 0.0002:
    rmses = []
    for feature in features:
        xgbreg = XGBRegressor(
            max_depth=9,
            learning_rate=0.08,
            n_estimators=3000,
            random_state=22,
        )
        
        xgbreg.fit(X_train[opt_features+feature], y_train, verbose=False,
                   eval_set=[(X_val[features+feature], y_val)],
                   early_stopping_rounds=7)
        
        y_predict = xgbreg.predict(X_val)
        rmse = mean_squared_error(y_val, y_predict)**0.5
        rmses.append(rmse)
    
    best_rmse_ind = np.argmax(rmses)
    current_rmse = rmses[best_rmse_ind]
    current_rmses.append(current_rmse)
    opt_features.append(features[best_rmse_ind])
    features.drop(features[best_rmse_ind])

    print(current_rmses)
    print(opt_features)

NameError: name 'XGBRegressor' is not defined